In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


In [9]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [15]:
tokens = tokenizer.encode('It was best but couldve been better. good', return_tensors='pt')

In [16]:
result = model(tokens)

In [17]:
result.logits

tensor([[ 1.9085,  2.2926,  1.2970, -1.6393, -3.1859]],
       grad_fn=<AddmmBackward0>)

In [18]:
int(torch.argmax(result.logits))+1

2

In [22]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2?osq=mejico+sydney')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
df['review'].iloc[0]

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
sentiment_score(df['review'].iloc[1])

2

In [29]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [30]:
df

,review,sentiment
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,Don't come here expecting legit Mexican food b...,3
4,Out of all the restaurants that I tried in Syd...,5
5,The food and service here was really good. It...,5
6,Visiting from Texas and decided to give this r...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4


In [31]:
df['review'].iloc[3]

"Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someone down to pay the bill. The watermelon salad was tasty but not complex, tossed with a few cucumbers and pistachios. The corn lollipops with spicy mayo sauce were probably the best dish. The beef empanadas were cold and average though the salsa was an interesting pickled onion. Definitely skip the octopus dish- it was dry, over cooked and lacked flavor though the presentation looked promising. It came with squares of polenta and dallops of potato mash with a dramatic squid ink streak across the dish bu